In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

import os
import time
import re

import tensorflow as tf
from tensorflow.python.platform import flags
import numpy as np
import keras
from keras import backend
import pandas as pd 

from cleverhans.attacks import FastGradientMethod
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.train import train
from cleverhans.utils import AccuracyReport
from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper
from cleverhans.utils_tf import model_eval
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from cleverhans.attacks import CarliniWagnerL2

import glob
FLAGS = flags.FLAGS

NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = .001
TRAIN_DIR = 'train_dir'
FILENAME = 'mnist.ckpt'
LOAD_MODEL = False



/Users/bibinsebastian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
classifier = keras.models.load_model("64_new_cropped_model_15_epo.h5")
import random
random.seed = 99

In [3]:
# # Args
# FLAGS = flags.FLAGS

# VIZ_ENABLED = True
# BATCH_SIZE = 128
# NB_EPOCHS = 6
# SOURCE_SAMPLES = 10
# LEARNING_RATE = .001
# CW_LEARNING_RATE = .005
# ATTACK_ITERATIONS = 100
# MODEL_PATH = ''
# TARGETED = False

In [4]:
def cw_attack(attack_iterations,src,CW_LEARNING_RATE,folder_num):
    """
    MNIST CleverHans tutorial
    :param train_start: index of first training set example
    :param train_end: index of last training set example
    :param test_start: index of first test set example
    :param test_end: index of last test set example
    :param nb_epochs: number of epochs to train model
    :param batch_size: size of training batches
    :param learning_rate: learning rate for training
    :param train_dir: Directory storing the saved model
    :param filename: Filename to save model under
    :param load_model: True for load, False for not load
    :param testing: if true, test error is calculated
    :param label_smoothing: float, amount of label smoothing for cross entropy
    :return: an AccuracyReport object
    """
    keras.layers.core.K.set_learning_phase(0)


    # Set TF random seed to improve reproducibility
    tf.set_random_seed(1234)

    if not hasattr(backend, "tf"):
        raise RuntimeError("This tutorial requires keras to be configured"
                           " to use the TensorFlow backend.")

    if keras.backend.image_dim_ordering() != 'tf':
        keras.backend.set_image_dim_ordering('tf')
        print("INFO: '~/.keras/keras.json' sets 'image_dim_ordering' to "
              "'th', temporarily setting to 'tf'")

    img_folder = src+'/*/*.jpg'
    
    dest_folder = 'CW_Phase'+str(folder_num)+'CW_LR'+str(CW_LEARNING_RATE)    
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    m = dest_folder + '/Male'
    f = dest_folder + '/Female'
    if not os.path.exists(m):
        os.makedirs(m)
    if not os.path.exists(f):
        os.makedirs(f)
    
        
    dest_folder = dest_folder
    

    # Change address
    addrs = glob.glob(img_folder)
    image =[]
    for add in addrs:
      img = cv2.imread(add);
      img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = img /255
      image.append(img)

    x_celeba = np.array(image)
    # Obtain Image Parameters

    #ADD RIGHT DATA 
    img_rows, img_cols, nchannels = x_celeba.shape[1:4]

    #MALE OR FEMALE 
    nb_classes = 1

    # Define input TF placeholder
    x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
    y = tf.placeholder(tf.float32, shape=(None, nb_classes))


    keras_wrapped_model = KerasModelWrapper(classifier)

    # get session from the model
    sess =keras.backend.get_session()
    # create attack object
    adv_ys = None
    yname = "y"
    VIZ_ENABLED = True
    BATCH_SIZE = 128
    NB_EPOCHS = 6
    SOURCE_SAMPLES = 10
    LEARNING_RATE = .001
    ATTACK_ITERATIONS = attack_iterations

    cw = CarliniWagnerL2(keras_wrapped_model, sess=sess)

    cw_params = {'binary_search_steps': 1,
               yname: adv_ys,
               'max_iterations': ATTACK_ITERATIONS,
               'learning_rate': CW_LEARNING_RATE,
               'batch_size': 10,
               'initial_const': 10}

    adv_x = cw.generate_np(x_celeba,**cw_params)


    # Consider the attack to be constant
    adv_x = tf.stop_gradient(adv_x) 
    attack_images = adv_x.eval(session =sess)
    y = np.concatenate([np.ones(50),np.zeros(50)])

    
    adv_pred = classifier.predict_classes(attack_images)
    # accuracy score
    
    acc = accuracy_score(y, adv_pred)
    adv_prob = classifier.predict_proba(attack_images)
    probcsv = dest_folder + '/Pred_prob.csv' 
    
    
    df = pd.DataFrame(adv_prob)
    df = pd.concat([pd.Series(addrs),df],axis=1)
    df.to_csv(probcsv)
    
    reg = src
    
    adv_addrs = [re.sub(reg,dest_folder,x) for x in addrs]
    
    
    for i, a in enumerate(attack_images):
        plt.imsave(adv_addrs[i],a)


    return(acc)


In [5]:
attack_iterations = 100
CW_LR = [0.0005,0.003,0.005,0.009,0.02,0.04,0.07]
img_folder = 'clean'

In [6]:
accuracy = []
for i,CW_LEARNING_RATE in enumerate(CW_LR):
    start = time.time()
    acc = cw_attack(100,img_folder,CW_LEARNING_RATE,i+1)
    end = time.time()
    accuracy.append(acc)
    print('Accuracy for {}'.format(CW_LEARNING_RATE), accuracy[i])
    print("Total time for {}".format(CW_LEARNING_RATE), (start-end))


[INFO 2018-11-25 00:07:34,749 cleverhans] Constructing new graph for attack CarliniWagnerL2
[INFO 2018-11-25 01:01:54,333 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy for 0.0005 0.17
Total time for 0.0005 -3259.6189348697662


[INFO 2018-11-25 01:19:07,419 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy for 0.003 0.0
Total time for 0.003 -1033.113486289978


[INFO 2018-11-25 01:30:55,393 cleverhans] Constructing new graph for attack CarliniWagnerL2


...

Accuracy for 0.009 0.0
Total time for 0.009 -620.1328999996185


[INFO 2018-11-25 02:06:25,858 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy for 0.02 0.0
Total time for 0.02 -1510.3357229232788


[INFO 2018-11-25 02:34:11,805 cleverhans] Constructing new graph for attack CarliniWagnerL2


Accuracy for 0.04 0.0
Total time for 0.04 -1665.9499051570892
Accuracy for 0.07 0.0
Total time for 0.07 -1511.0610539913177


In [7]:
c= np.vstack((CW_LR,accuracy))
c

array([[0.0005, 0.003 , 0.005 , 0.009 , 0.02  , 0.04  , 0.07  ],
       [0.17  , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ]])

In [8]:
report = pd.DataFrame(c.T,columns=['Learning_Rate','Accuracy'])

In [9]:
report.to_csv('CW_attack_accuracy.csv')